In [1]:
%matplotlib inline
import re
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from joblib import Parallel, delayed
from tqdm.notebook import tqdm as tqdm
from tqdm.notebook import trange
import contextlib
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import edward2 as ed
import tensorflow as tf
from scipy.special import digamma
from pickle import dump, load
from scipy.sparse import csr_matrix
import tensorflow_probability as tfp
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import os
import sys
import time

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# tensorflow does not work with new numpy versions
assert np.__version__  < '1.20'



/home/iron/Repositories/classes/ml_2_classes/ML2_Exercises/pgm/ex7/denys/LDA_nips_papers/.env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
tqdm.pandas()

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/iron/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# data_proc = pd.read_pickle('data_proc.pkl')
# data_enc = pd.read_pickle('data.pkl')
# with open('word_to_idx.pkl', 'rb') as f:
#     words_to_idx = load(f)

In [5]:
seed = 42
data = pd.read_csv('nips-papers/papers.csv')
data = data[['paper_text']]
data_n = data.sample(n=100, random_state=42)

In [6]:
# data_n.head()

In [7]:
def func(text):
    text = text.lower()
    text = re.sub(r'(\d+)', '', text)
    text = re.sub(r'(\n)|(\t)', ' ', text)
    text = text.translate({ ord(c): None for c in string.punctuation })
    text = text.strip()
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    text = [i for i in tokens if not i in stop_words and len(i) > 1]
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

In [8]:
data_proc = data_n['paper_text'].progress_apply(func)

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
data_proc.head()

509     [independent, component, analysis, identificat...
2576    [nearmaximum, entropy, model, binary, neural, ...
6362    [nearestneighbor, sample, compression, efficie...
6173    [efficient, highorder, interactionaware, featu...
6552    [multioutput, polynomial, network, factorizati...
Name: paper_text, dtype: object

In [10]:
# words_to_idx = {}
# idx_to_word = {}
# def encode(text, words_to_idx, idx_to_word):
#     text_enc = np.empty(len(text), dtype=int)
#     for i, word in enumerate(text):
#         if word not in words_to_idx:
#             idx = words_to_idx[word] = len(words_to_idx)
#             idx_to_word[idx] = word
#         else:
#             idx = words_to_idx[word]
#         text_enc[i] = idx
#     return text_enc
# data_enc = data_proc.progress_apply(encode, args=(words_to_idx, idx_to_word))

In [11]:
def encode2(text, word_dict):
    return np.asarray(word_dict.doc2idx(text))

word_dict = Dictionary(data_proc)
data_enc = data_proc.progress_apply(lambda x: encode2(x, word_dict))

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
# data_proc.to_pickle('data_proc.pkl')
# data_enc.to_pickle('data.pkl')

# with open('word_to_idx.pkl', 'wb') as f:
#     dump(words_to_idx, f)

In [13]:
K = 5
D = len(data_enc)
Ns = data_enc.apply(lambda x: len(x)).to_numpy().astype(np.int)
N = Ns.sum()
V = len(word_dict)

In [14]:
print(K, D, Ns, N, V)

5 100 [1473 2453 3310 3107 3017 1621 1727 1618 3434 1774 1454 1339 2501 1669
 1785 1426 1956 2017 2192 1927 2614 2839 1642 2324 1934 3103 3264 2957
 2776 1736 3298 2756 1244 1501 1855 1112 1979 2687 1972 2950 2946 2881
 2925 2306 2767 3010 1629 1468 2904 1105 1847 1282 1463 2645 2189 3190
 1907 1033 2197 1437 3537 2117 1484 2646 2227 2141 3498 1673 2113 2894
 2744 1479 1917 3173 2566 2640 2765 2451 2814 2804 1580 1656 1806 2439
 2567 2834 2827 2455 2306 1458 3056 2524 1648 2298 1153 3384 2860 3158
 2646 1954] 227766 18411


In [15]:
def create_sparse(data, alpha, D, K, V):
    Ns = np.empty(D, dtype=np.int)
     
    for i, doc in enumerate(data):
        Ns[i] = len(doc)
    
    N = Ns.sum()
    rows = np.empty(N, dtype=np.int64)
    cols = np.empty(N, dtype=np.int64)
    v_cols = np.empty(N, dtype=np.int64)
    
    last_idx = 0
    
    for i, doc in tqdm(enumerate(data), total=D):
        n = len(doc)
        rows[last_idx:last_idx+n] = i
        cols[last_idx:last_idx+n] = np.arange(n, dtype=np.int64)
        v_cols[last_idx:last_idx+n] = doc
        last_idx += n
        
    N_max = Ns.max()
    phi_rows = np.repeat(rows, K)
    phi_cols = np.repeat(cols, K)
    phi3 = np.tile(np.arange(K), len(rows))  
    phi_indices = np.vstack((phi_rows, phi_cols, phi3)).T
    phi = tf.sparse.SparseTensor(phi_indices, values=tf.fill((len(phi_indices),), 1/K), dense_shape=(D, N_max, K))
    phi = tf.sparse.reorder(phi)
    gamma_v_cols = np.repeat(v_cols, K)
    gamma = tf.expand_dims(alpha, 0) + (Ns / K).reshape(-1, 1)

    beta_indices = np.vstack((phi3, gamma_v_cols)).T
    gamma_indices = np.vstack((phi_rows, phi3)).T
    w_indices = np.vstack((phi_rows, phi_cols, phi3, gamma_v_cols)).T
    tmp = tf.sparse.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V))
    tmp = tf.sparse.reorder(tmp)
    w_indices = tmp.indices
    lmbd = tf.fill((K, V), 1/V)
    return phi, gamma, lmbd, tf.constant(beta_indices, dtype=tf.int64), tf.constant(gamma_indices, dtype=tf.int64), tf.constant(w_indices, dtype=tf.int64), N_max

In [ ]:
# alpha_n = np.random.rand(K).astype(np.float32)
# eta_n = np.random.rand(1).astype(np.float32)
# beta_n = np.random.dirichlet(np.random.rand(V).astype(np.float32), size=K).astype(np.float32)
# phi_n = [np.full((n, K), 1/K).astype(np.float32) for n in Ns]
# gamma_n = alpha_n.reshape(1, -1) + Ns.reshape(-1, 1) / K
# gammma_n = gamma_n.astype(np.float32)
# lmbd_n = np.full((K, V), eta_n)
# lmbd_n = lmbd_n.astype(np.float32)

In [16]:
# It tries to estimate also smoothed params, but eta tends to go negative, breaking all the stuff.
# Maybe loss is incorrect

class Positive(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return w * tf.cast(tf.math.greater(w, 0.), w.dtype)   
    
@tf.function
def calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V):
    digamma = tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True))
    ta = tf.math.lgamma(tf.math.reduce_sum(alpha)) - \
                                          tf.math.reduce_sum(tf.math.lgamma(alpha)) + \
                                          tf.math.reduce_sum(tf.expand_dims(alpha - 1, 0)*digamma, axis=1)
    zt = tf.sparse.reduce_sum(tf.expand_dims(digamma, 1)*phi, axis=(1, 2))
    wzb = tf.math.reduce_sum(tf.expand_dims(tf.math.log1p(beta), axis=0) * tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(1)), axis=(1, 2))
    bl = tf.math.reduce_sum((eta-1)*tf.math.reduce_sum(tf.math.digamma(lmbd) - tf.math.digamma(tf.math.reduce_sum(lmbd, axis=1, keepdims=True)), axis=1) + \
         tf.math.lgamma(eta*V) - V*tf.math.lgamma(eta))
    qt = -tf.math.lgamma(tf.math.reduce_sum(gamma, axis=1)) + tf.math.reduce_sum(tf.math.lgamma(gamma), axis=1) - tf.math.reduce_sum((gamma-1)*digamma, axis=1)
    phi_log_phi = tf.sparse.SparseTensor(phi.indices, phi.values*tf.math.log1p(phi.values), dense_shape=[D, N_max, K])
    qz = tf.sparse.reduce_sum(phi_log_phi, axis=(1, 2))
    elbo = tf.math.reduce_sum(ta + zt + qt + qz)
    return elbo

@tf.function
def e_step_it(alpha, beta_mod, eta, phi, gamma, lmbd, gamma_indices, w_indices, D, K, N_max, V):
    # phi
    dg = tf.math.exp(tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True)))
    phi = tf.sparse.SparseTensor(phi.indices, beta_mod*tf.gather_nd(dg, gamma_indices), dense_shape=[D, N_max, K])
    phi /= tf.sparse.reduce_sum(phi, axis=2, keepdims=True) + 1e-5
    # gamma
    gamma = tf.expand_dims(alpha, 0) + tf.sparse.reduce_sum(phi, axis=1)
    # lambda
    lmbd = eta + tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(0, 1))
    
    gamma.set_shape((D, K))
    lmbd.set_shape((K, V))
    return phi, gamma, lmbd

@tf.function
def e_step(alpha, beta, eta, phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, D, K, N_max, V, max_it=1000, rtol=1e-03, atol=1e-03):      
    beta_mod = tf.gather_nd(beta, beta_indices)

    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=rtol, atol=atol)
    tf_cond = lambda i, elbo_old, elbo, args: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def tf_body(i, elbo_old, elbo, args):
        phi, gamma, lmbd = args
        i = i + 1
        elbo_old = tf.identity(elbo)
        phi, gamma, lmbd = e_step_it(alpha, beta_mod, eta, phi, gamma, lmbd, gamma_indices, w_indices, D, K, N_max, V)
        elbo = calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V)
        args = (phi, gamma, lmbd)
        return (i, elbo_old, elbo, args)
    
    i = tf.constant(0, name='e_loop_counter')
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    args = (phi, gamma, lmbd)
    
    i, _, elbo, args = tf.while_loop(tf_cond, tf_body, [i, elbo_old, elbo, args], parallel_iterations=1)
    phi, gamma, lmbd = args
    return phi, gamma, lmbd

@tf.function
def m_step(alpha, alpha_2, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V, opt, max_it=1000, rtol=1e-03, atol=1e-03):
    # beta
    beta = tf.sparse.reduce_sum(tf.SparseTensor(w_indices, phi.values, dense_shape=(D, N_max, K, V)), axis=(0, 1))
    beta /= tf.math.reduce_sum(beta, axis=1, keepdims=True) + 1e-5
    # alpha
    digamma = tf.math.reduce_sum(tf.math.digamma(gamma) - tf.math.digamma(tf.math.reduce_sum(gamma, axis=1, keepdims=True)), axis=0)
    
    @tf.function
    def alpha_elbo(alpha, digamma):
        val = tf.math.reduce_sum(tf.math.lgamma(tf.math.reduce_sum(alpha)) - \
                                          tf.math.reduce_sum(tf.math.lgamma(alpha)) + \
                                          tf.math.reduce_sum(tf.expand_dims(alpha - 1, 0)*digamma, axis=1))
        return val
    
    alpha_it_cond = lambda i, digamma: i < K
    
    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=rtol, atol=atol)
    alpha_cond = lambda i, elbo_old, elbo, digamma: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def alpha_it_body(i, digamma):        
        def loss_func():
            ta = tf.TensorArray(alpha.dtype, size=K, dynamic_size=False)
            alpha_copy = ta.unstack(alpha_2)
            alpha_copy = alpha_copy.write(i, tf.math.maximum(alpha[i], 1e-5))
            alpha_copy = alpha_copy.stack()
            alpha.assign(alpha_copy)
            alpha_2.assign(alpha_copy)
            a = tf.math.reduce_sum((alpha - 1)*digamma)
            res = -(D*(tf.math.lgamma(tf.math.reduce_sum(alpha)) - tf.math.reduce_sum(tf.math.lgamma(alpha))) + a)
            return res
        losses = tfp.math.minimize(loss_func, max_it, opt, convergence_criterion=tfp.optimizer.convergence_criteria.LossNotDecreasing(rtol=rtol, atol=atol), 
                                   trainable_variables=[alpha])
        
        ta = tf.TensorArray(alpha.dtype, size=K, dynamic_size=False)
        alpha_copy = ta.unstack(alpha_2)
        alpha_copy = alpha_copy.write(i, tf.math.maximum(alpha[i], 1e-5))
        alpha_copy = alpha_copy.stack()
        alpha.assign(alpha_copy)
        
        return i + 1, digamma
    
    @tf.function
    def alpha_body(i, elbo_old, elbo, digamma):
        i = i + 1
        j = tf.constant(0)
        elbo_old = tf.identity(elbo)
        j, _ = tf.while_loop(alpha_it_cond, alpha_it_body, [j, digamma], parallel_iterations=1)
#         tf.print(alpha, output_stream=sys.stdout)
        elbo = alpha_elbo(alpha, digamma)
        return (i, elbo_old, elbo, digamma)
    
    i = tf.constant(0)
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    i, _, elbo, _ = tf.while_loop(alpha_cond, alpha_body, [i, elbo_old, elbo, digamma], parallel_iterations=1)
    
    di_lmbd = tf.math.reduce_sum(tf.math.digamma(lmbd) - tf.math.digamma(tf.math.reduce_sum(lmbd, axis=1, keepdims=True))) 
    # eta
    def loss_func():
        loss_v = -((eta-1)*di_lmbd + K*(tf.math.lgamma(eta*V) - V*tf.math.lgamma(eta)))
        return loss_v
    
    losses = tfp.math.minimize(loss_func, max_it, opt, convergence_criterion=tfp.optimizer.convergence_criteria.LossNotDecreasing(rtol=rtol, atol=atol), trainable_variables=[eta])
    eta.assign(tf.math.maximum(eta, 0.1))
    return beta

def train(data, D, K, V, alpha=None, eta=None, max_it=1000, seed=42, rtol=1e-3, atol=1e-3):
    """
    alpha
        The alpha prior. Must be a single value
    eta
        The eta prior. Must be a single value
    """
    start = time.time()
    if alpha:
        alpha = tf.zeros((K,)) + alpha
    else:
        alpha = tf.random.uniform((K,))
    
    if eta:
        eta = tf.zeros((1,)) + eta
    else:
        eta = tf.random.uniform((1,))
        
    beta = tf.convert_to_tensor(np.random.dirichlet(np.zeros((V,)) + eta, size=K), dtype=tf.float32)

    opt = tf.optimizers.Adam(1e-3)
    
    phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, N_max = create_sparse(data, alpha, D, K, V)
    N_max = N_max.item()
    
    alpha = tf.Variable(alpha, constraint=Positive())
    alpha_2 = tf.Variable(tf.identity(alpha), trainable=False)
    eta = tf.Variable(eta, constraint=Positive())

    np_isclose = lambda elbo_old, elbo: np.allclose(elbo_old, elbo, rtol=1e-3, atol=1e-3)
    train_cond = lambda i, elbo_old, elbo, args: tf.logical_and(i < max_it, tf.logical_or(i == 0, tf.logical_not(tf.numpy_function(np_isclose, [elbo_old, elbo], tf.bool))))
    
    @tf.function
    def train_body(i, elbo_old, elbo, args):
        i = i + 1
        beta, phi, gamma, lmbd = args
        elbo_old = elbo
        tf.print(i, ': E step', output_stream=sys.stdout, end='\t')
        phi, gamma, lmbd = e_step(alpha, beta, eta, phi, gamma, lmbd, beta_indices, gamma_indices, w_indices, D, K, N_max, V, atol=atol, rtol=rtol)
        tf.print('M step', output_stream=sys.stdout, end='\t')
        beta = m_step(alpha, alpha_2, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V, opt, atol=atol, rtol=rtol)
        elbo = calc_elbo(alpha, beta, eta, phi, gamma, lmbd, w_indices, D, K, N_max, V)
        tf.print('ELBO =', elbo, output_stream=sys.stdout)
        args = (beta, phi, gamma, lmbd)
        return i, elbo_old, elbo, args
    
    i = tf.constant(0)
    args = (beta, phi, gamma, lmbd)
    elbo = tf.constant(np.inf, dtype=tf.float32)
    elbo_old = tf.identity(elbo)
    print('Preparing...')
    i, elbo_old, elbo, args = tf.while_loop(train_cond, train_body, [i, elbo_old, elbo, args], parallel_iterations=1)
    beta, phi, gamma, lmbd = args
    tf.print('Converged in', i,  'iterations', output_stream=sys.stdout)
    end = time.time()
    print('Time:', end-start, 's')
    return alpha, beta, eta, phi, gamma, lmbd, elbo

alpha, beta, eta, phi, gamma, lmbd, elbo = train(data_enc, D, K, V)

  0%|          | 0/100 [00:00<?, ?it/s]

Preparing...
1 : E step	M step	ELBO = -233279.766
2 : E step	M step	ELBO = -236907.844
3 : E step	M step	ELBO = -197734.438
4 : E step	M step	ELBO = -151478.844
5 : E step	M step	ELBO = -109258.789
6 : E step	M step	ELBO = -76375.2578
7 : E step	M step	ELBO = -49924.5977
8 : E step	M step	ELBO = -30471.8262
9 : E step	M step	ELBO = -15996.1758
10 : E step	M step	ELBO = -3363.90479
11 : E step	M step	ELBO = 8155.35498
12 : E step	M step	ELBO = 20044.6816
13 : E step	M step	ELBO = 29865.5039
14 : E step	M step	ELBO = 37720.6133
15 : E step	M step	ELBO = 45035.4297
16 : E step	M step	ELBO = 52159.8203
17 : E step	M step	ELBO = 58456.8672
18 : E step	M step	ELBO = 63638.8086
19 : E step	M step	ELBO = 67640.2344
20 : E step	M step	ELBO = 70649.4375
21 : E step	M step	ELBO = 72807.2891
22 : E step	M step	ELBO = 74468.4688
23 : E step	M step	ELBO = 75988.7109
24 : E step	M step	ELBO = 77478.2109
25 : E step	M step	ELBO = 78874.4531
26 : E step	M step	ELBO = 80053.2344
27 : E step	M step	ELBO 

In [ ]:
# # Generate doc
# theta_d = ed.Dirichlet(alpha)
# for i in range(100):
#     z_dn = ed.Categorical(probs=theta_d)
#     w_dn = ed.Categorical(probs=beta[z_dn])
#     print(word_dict[w_dn.numpy()])


In [17]:
# Get topic top words
def get_topics(beta, word_dict, k):
    topics = []
    weights = []
    for h in range(k):
        top_beta = tf.math.top_k(beta[h], k=10)
        topic = [word_dict[i] for i in top_beta[1].numpy()] 
        weight = list(top_beta[0].numpy())
    #     print(([idx_to_word[i] for i in tf.math.top_k(beta[h], k=10)[1].numpy()]))
        topics.append(topic)
        weights.append(weight)

    return (np.asarray(topics), np.asarray(weights))

topics, weights = get_topics(beta, word_dict, K)
topics

array([['algorithm', 'data', 'graph', 'problem', 'learning', 'method',
        'function', 'set', 'matrix', 'model'],
       ['feature', 'game', 'image', 'algorithm', 'set', 'strategy',
        'player', 'action', 'method', 'model'],
       ['function', 'model', 'distribution', 'data', 'learning',
        'algorithm', 'probability', 'sample', 'set', 'problem'],
       ['neuron', 'model', 'neural', 'network', 'input', 'figure',
        'weight', 'cell', 'learning', 'signal'],
       ['network', 'model', 'neural', 'data', 'learning', 'training',
        'parameter', 'number', 'using', 'set']], dtype='<U12')

In [18]:
def get_coherence(topics, text_data, word_dict, coherence_type='c_v'):
    coherence_model_lda = CoherenceModel(topics=topics, texts=text_data, dictionary=word_dict, coherence='c_v')
    return coherence_model_lda.get_coherence()

coherence = get_coherence(topics, data_proc, word_dict, 'c_v')
coherence

0.4213764018817706

Parameter exploration loop

In [ ]:
Ns = [25, 50]
Ks = range(1, 15, 3)
etas = list(np.arange(0.01, 1., 0.3)) + [5, 10]
alphas = list(np.arange(0.01, 1., 0.3)) + [5, 10]

params = []
for n in Ns:
    n_params = []
    for k in Ks:
        for a in alphas:
            for e in etas:
                n_params.append((k, a, e))
    params.append((n, n_params))

results = pd.DataFrame(columns=["N", "K", "alpha", "eta", "elbo", "coherence", "topics", "weights"])

i = 0
for n, n_params in tqdm(params):
    data_n = data.sample(n=n, random_state=42)
    data_proc = data_n['paper_text'].progress_apply(func)
    word_dict = Dictionary(data_proc)
    data_enc = data_proc.progress_apply(lambda x: encode2(x, word_dict))
    V = len(word_dict)
    
    for k, a, e in tqdm(n_params):
        print(n, k, a, e)
        _, beta, _, _, _, _, elbo = train(data_enc, n, k, V, alpha=a, eta=e)
        topics, weights = get_topics(beta, word_dict, k=k)
        coherence = get_coherence(topics, data_proc, word_dict, 'c_v')
        str_topics = repr([list(topic) for topic in topics])
        str_weights = repr([list(weight) for weight in weights])
        results.loc[i] = n, k, a, e, elbo.numpy(), coherence, str_topics, str_weights
        i += 1

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

25 1 0.01 0.01


...

Preparing...
1 : E step	M step	ELBO = -7407.19238
2 : E step	M step	ELBO = -13131.4414
3 : E step	M step	ELBO = -16784.1934
4 : E step	M step	ELBO = -20101.418
5 : E step	M step	ELBO = -22259.7129
6 : E step	M step	ELBO = -24007.6016
7 : E step	M step	ELBO = -25318.0879
8 : E step	M step	ELBO = -26879.4199
9 : E step	M step	ELBO = -28083.0137
10 : E step	M step	ELBO = -28943.9668
11 : E step	M step	ELBO = -29519.1387
12 : E step	M step	ELBO = -30109.6699
13 : E step	M step	ELBO = -30822.0312
14 : E step	M step	ELBO = -31233.7969
15 : E step	M step	ELBO = -31567.4102
16 : E step	M step	ELBO = -31891.2852
17 : E step	M step	ELBO = -32142.5781
18 : E step	M step	ELBO = -32308.791
19 : E step	M step	ELBO = -32467.6777
20 : E step	M step	ELBO = -32629.2871
21 : E step	M step	ELBO = -32736.0527
22 : E step	M step	ELBO = -32822.2734
23 : E step	M step	ELBO = -32904.3594
24 : E step	M step	ELBO = -32983.2656
25 : E step	M step	ELBO = -33042.0391
26 : E step	M step	ELBO = -33085.8164
27 : E ste

  0%|          | 0/25 [00:00<?, ?it/s]

Preparing...
1 : E step	M step	ELBO = -43157.7734
2 : E step	M step	ELBO = -45859.0117
3 : E step	M step	ELBO = -43997.0898
4 : E step	M step	ELBO = -41315.8633
5 : E step	M step	ELBO = -37775.875
6 : E step	M step	ELBO = -33570.3555
7 : E step	M step	ELBO = -28916.6387
8 : E step	M step	ELBO = -23927.5273
9 : E step	M step	ELBO = -18956.9785
10 : E step	M step	ELBO = -13753.8281
11 : E step	M step	ELBO = -8461.61719
12 : E step	M step	ELBO = -3447.50635
13 : E step	M step	ELBO = 1081.01208
14 : E step	M step	ELBO = 4816.12
15 : E step	M step	ELBO = 8283.4
16 : E step	M step	ELBO = 11779.9033
17 : E step	M step	ELBO = 14969.0957
18 : E step	M step	ELBO = 17383.2656
19 : E step	M step	ELBO = 19384.8438
20 : E step	M step	ELBO = 21124.332
21 : E step	M step	ELBO = 22495.8027
22 : E step	M step	ELBO = 23729.1152
23 : E step	M step	ELBO = 24684.7578
24 : E step	M step	ELBO = 25558.918
25 : E step	M step	ELBO = 26305.2793
26 : E step	M step	ELBO = 26855.7207
27 : E step	M step	ELBO = 27372.

  0%|          | 0/25 [00:00<?, ?it/s]

Preparing...
1 : E step	M step	

In [ ]:
results

In [ ]:
results.to_csv("results/resultset_1.csv", index=False)

In [ ]:
# MAX_IT = 10
# EPS = 0.001
# tf.executing_eagerly()
# optim = tf.keras.optimizers.Adam(1e-3)
# # B = ed.Dirichlet(concentration=tf.fill([K, V], 0.1), name="topics")
# # Z = ed.DirichletMultinomial(tf.convert_to_tensor(Ns), concentration=tf.fill([D, K], 0.1))
# alpha = np.copy(alpha_n).astype(np.float32)
# eta = np.copy(eta_n).astype(np.float32)

# beta = np.copy(beta_n).astype(np.float32)
# phi = [np.full((n, K), 1/K).astype(np.float32) for n in Ns]
# gamma = np.copy(gamma_n).astype(np.float32)
# lmbd = np.copy(lmbd_n).astype(np.float32)

# bb = None
# gg = None
# ww = None

# class Positive(tf.keras.constraints.Constraint):
#     def __call__(self, w):
#         return w * tf.cast(tf.math.greater(w, 0.), w.dtype)

# bb = []
# for it in trange(MAX_IT):
#     bb2 = []
#     gg2 = []
#     ww2 = []
#     print('before', gamma)
#     for d in range(D):
#         for n in range(Ns[d]):
#             for i in range(K):
#                 phi[d][n, i] = beta[i, data_enc.iloc[d][n]] * np.exp(digamma(gamma[d, i]) - digamma(np.sum(gamma[d])))
#         phi[d] /= np.sum(phi[d], axis=-1, keepdims=True) + 1e-5
        
#         for i in range(K):
#             gamma[d, i] = alpha[i] + np.sum(phi[d][:, i])
#     print('after', gamma)

#     lmbd = np.full((K, V), eta)
#     for i in range(K):
#         for j in range(V):
#             for d in range(D):
#                 mask = (data_enc.iloc[d] == j)
#                 lmbd[i, j] += np.sum(phi[d][:, i]*mask)
    
     
# #     if bb is None:
# #         bb = bb2
# #         gg = gg2
# #         ww = ww2
# #         break
                
#     alpha_t = tf.Variable(alpha, trainable=True, constraint=Positive())
#     gamma_t = tf.convert_to_tensor(gamma, dtype=tf.float32)
    
    
#     def f_x():
#         g_term = tf.math.reduce_sum(tf.expand_dims((alpha_t - 1), 0)*(tf.math.digamma(gamma_t) - 
#                                                    tf.math.digamma(tf.math.reduce_sum(gamma_t, axis=1, keepdims=True))), axis=1)
#         loss = -tf.math.reduce_sum(tf.math.lgamma(tf.math.reduce_sum(alpha_t)) - tf.math.reduce_sum(tf.math.lgamma(alpha_t)) + g_term)
#         return loss
    
#     for itt in range(10):
#         for i in range(K):
#             for itt1 in range(50):
#                 #with tf.GradientTape() as tape:
#                 optim.minimize(f_x, [alpha_t])
# #                 grads = tape.gradient(loss, opt_a)
# #                 optim.apply_gradients([(grads, opt_a)])
#                 alpha[i] = alpha_t.numpy()[i]
#                 np.nan_to_num(alpha, copy=False, nan=1e-5)
#                 alpha_t.assign(alpha)
#         print(alpha_t)
#     beta = (lmbd - eta) / (np.sum(lmbd - eta, axis=-1, keepdims=True) + 1e-5)
#     break
    
#     eta_t = tf.Variable(eta, trainable=True, constraint=Positive())
    
#     @tf.function
#     def f_eta():
#         loss = K*((eta_t-1)*(tf.math.digamma(eta_t) - tf.math.digamma(eta_t*V)) + tf.math.lgamma(eta_t*V) - V*tf.math.lgamma(eta))
#         return loss
    
#     for itt1 in range(50):
#         optim.minimize(f_eta, [eta_t])
#     eta = eta_t.numpy()

    